# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1485895731.log


This non-commercial license of GraphLab Create for academic use is assigned to n16144507@cs.hacettepe.edu.tr and will expire on October 30, 2017.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.634776     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.678796     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.get('coefficients').print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
import numpy as np
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [34]:
l1_penalties =  np.logspace(1, 7, num=13)
rss_values = []
for penalty in l1_penalties:
    model_new = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=penalty, verbose = False)
    
    results = model_new.predict(validation)
    residuals = results - validation['price']
    residuals_sq = residuals * residuals
    RSS = residuals_sq.sum()
    rss_values.append(RSS)
    print "for penalty:" , penalty, " RSS : " , RSS

index = rss_values.index(min(rss_values))

print l1_penalties[a]

for penalty: 10.0  RSS :  6.25766285142e+14
for penalty: 31.6227766017  RSS :  6.25766285362e+14
for penalty: 100.0  RSS :  6.25766286058e+14
for penalty: 316.227766017  RSS :  6.25766288257e+14
for penalty: 1000.0  RSS :  6.25766295212e+14
for penalty: 3162.27766017  RSS :  6.25766317206e+14
for penalty: 10000.0  RSS :  6.25766386761e+14
for penalty: 31622.7766017  RSS :  6.25766606749e+14
for penalty: 100000.0  RSS :  6.25767302792e+14
for penalty: 316227.766017  RSS :  6.25769507644e+14
for penalty: 1000000.0  RSS :  6.25776517727e+14
for penalty: 3162277.66017  RSS :  6.25799062845e+14
for penalty: 10000000.0  RSS :  6.25883719085e+14
10.0


In [28]:

model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10, verbose = False)

model.get('coefficients').print_rows(num_rows = 18)

[625766285142460.5, 625766285362394.4, 625766286057885.1, 625766288257224.4, 625766295212186.1, 625766317206080.8, 625766386760658.0, 625766606749278.5, 625767302791634.9, 625769507643885.8, 625776517727024.5, 625799062845466.6, 625883719085425.0]
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterf

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [26]:
print l1_penalties[a]
print a

10.0
0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [35]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [46]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [47]:
non_zeros = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)

    coeffs = model['coefficients']['value']
    non_zero = coeffs.nnz()
    non_zeros.append(non_zero)
    print "For l1_penalty ", l1_penalty, " non-zero weights: " , non_zero
    

For l1_penalty  100000000.0  non-zero weights:  18
For l1_penalty  127427498.57  non-zero weights:  18
For l1_penalty  162377673.919  non-zero weights:  18
For l1_penalty  206913808.111  non-zero weights:  18
For l1_penalty  263665089.873  non-zero weights:  17
For l1_penalty  335981828.628  non-zero weights:  17
For l1_penalty  428133239.872  non-zero weights:  17
For l1_penalty  545559478.117  non-zero weights:  17
For l1_penalty  695192796.178  non-zero weights:  17
For l1_penalty  885866790.41  non-zero weights:  16
For l1_penalty  1128837891.68  non-zero weights:  15
For l1_penalty  1438449888.29  non-zero weights:  15
For l1_penalty  1832980710.83  non-zero weights:  13
For l1_penalty  2335721469.09  non-zero weights:  12
For l1_penalty  2976351441.63  non-zero weights:  10
For l1_penalty  3792690190.73  non-zero weights:  6
For l1_penalty  4832930238.57  non-zero weights:  5
For l1_penalty  6158482110.66  non-zero weights:  3
For l1_penalty  7847599703.51  non-zero weights:  1
F

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [14]:
print non_zeros
print max_nonzeros

greater = list(l1_penalty_values[0:14])
smaller = list(l1_penalty_values[15:])

print greater
print smaller

print max(greater)
print min(smaller)

l1_penalty_min =max(greater) 
l1_penalty_max = min(smaller)

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]
7
[100000000.0, 127427498.57031322, 162377673.91887242, 206913808.11147901, 263665089.87303555, 335981828.62837881, 428133239.8719396, 545559478.11685145, 695192796.17755914, 885866790.41008317, 1128837891.6846883, 1438449888.2876658, 1832980710.8324375, 2335721469.0901213]
[3792690190.7322536, 4832930238.5717525, 6158482110.6602545, 7847599703.5146227, 10000000000.0]
2335721469.09
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [40]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [42]:
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    
    results = model.predict(validation)
    residuals = results - validation['price']
    residuals_sq = residuals * residuals
    RSS = residuals_sq.sum()
    rss_values.append(RSS)
    
    coeffs = model['coefficients']['value']
    non_zero = coeffs.nnz()
    
    print "l1_penalty: " , l1_penalty, ", RSS: ", RSS, ", Sparsity: ", non_zero
    
print rss_values

l1_penalty:  2335721469.09 , RSS:  8.69018172894e+14 , Sparsity:  12
l1_penalty:  2412404033.39 , RSS:  8.8129834303e+14 , Sparsity:  11
l1_penalty:  2489086597.68 , RSS:  8.93028822817e+14 , Sparsity:  11
l1_penalty:  2565769161.98 , RSS:  9.05027717563e+14 , Sparsity:  11
l1_penalty:  2642451726.28 , RSS:  9.17301120552e+14 , Sparsity:  11
l1_penalty:  2719134290.57 , RSS:  9.29711455616e+14 , Sparsity:  10
l1_penalty:  2795816854.87 , RSS:  9.40117561994e+14 , Sparsity:  10
l1_penalty:  2872499419.17 , RSS:  9.50707115518e+14 , Sparsity:  10
l1_penalty:  2949181983.47 , RSS:  9.62478692595e+14 , Sparsity:  10
l1_penalty:  3025864547.76 , RSS:  9.75107157856e+14 , Sparsity:  10
l1_penalty:  3102547112.06 , RSS:  9.88808783979e+14 , Sparsity:  10
l1_penalty:  3179229676.36 , RSS:  1.00576865809e+15 , Sparsity:  10
l1_penalty:  3255912240.65 , RSS:  1.02332388595e+15 , Sparsity:  10
l1_penalty:  3332594804.95 , RSS:  1.03576201921e+15 , Sparsity:  8
l1_penalty:  3409277369.25 , RSS:  1

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [17]:
rss_values.index(min(rss_values))


0

In [43]:
3485959933.54

3485959933.54

In [44]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3485959933.54, verbose = False)

In [45]:
model.get('coefficients').print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 224230.647176 |  None  |
|     bedrooms     |  None | 521.298855292 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15675.2855172 |  None  |
|   sqft_living    |  None | 32.2348252764 |  None  |
| sqft_living_sqrt |  None | 680.566539524 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2836.30618613 |  None  |
|    sqft_above    |  None | 29.7586306058 |  None  |
|  sqft_basement   |  None |